In [1]:
import time
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import *
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import joblib
from sklearn.preprocessing import MultiLabelBinarizer
import streamlit as st

In [2]:
Data = joblib.load('dfcp.sav')

In [3]:
df = Data.copy()

In [4]:
df

,Title,Body,Tags,Body_words,Title_words,Body_words_join,Title_words_join,SL_Body,SL_Title
0,Qt FullScreen on Startup,I want to start an application on fullscreen (...,<c++>,"[start, application, fullscreen, MacOS, 10, 8,...","[Qt, FullScreen, Startup]",start application fullscreen MacOS 10 8 x Qt d...,Qt FullScreen Startup,"[start, applic, fullscreen, maco, 10, 8, x, qt...","[qt, fullscreen, startup]"
1,jQuery or Javascript - how to disable window s...,Hi is it possible to disable window scrolling ...,<javascript><jquery><css>,"[Hi, possible, disable, window, scrolling, wit...","[Javascript, disable, window, scroll, overflow...",Hi possible disable window scrolling without o...,Javascript disable window scroll overflow hidden,"[hi, possibl, disabl, window, scroll, without,...","[javascript, disabl, window, scroll, overflow,..."
2,Python mock: mocking base class for inheritance,I am testing a class that inherits from anothe...,<python>,"[testing, inherits, another, complex, DB, conn...","[mock, mocking, base, inheritance]",testing inherits another complex DB connection...,mock mocking base inheritance,"[test, inherit, anoth, complex, db, connect, m...","[mock, mock, base, inherit]"
3,Calling CreateProcessAsUser from C#,I've been attempting to create a new process u...,<c#>,"[attempting, create, process, context, specifi...","[Calling, CreateProcessAsUser]",attempting create process context specific Cre...,Calling CreateProcessAsUser,"[attempt, creat, process, context, specif, cre...","[call, createprocessasus]"
4,How does Facebook keep the header and footer f...,When browsing through Facebook pages the heade...,<javascript><html>,"[When, browsing, Facebook, pages, header, fixe...","[Facebook, keep, header, footer, fixed, loadin...",When browsing Facebook pages header fixed foot...,Facebook keep header footer fixed loading diff...,"[when, brow, facebook, page, header, fix, foot...","[facebook, keep, header, footer, fix, load, di..."
...,...,...,...,...,...,...,...,...,...
49994,Why would you choose the Java programming lang...,Why would you choose java over others? Why did...,<java>>,"[Why, choose, others, Why, choose, program, ap...","[would, choose, programming, language, others]",Why choose others Why choose program applicati...,would choose programming language others,"[whi, choos, other, whi, choos, program, appli...","[would, choos, program, languag, other]"
49995,How do I do a deep copy of an element in LINQ ...,I want to make a deep copy of a LINQ to XML XE...,<.net>,"[make, deep, copy, LINQ, XML, XElement, reason...","[deep, copy, element, LINQ, XML]",make deep copy LINQ XML XElement reason nodes ...,deep copy element LINQ XML,"[make, deep, copi, linq, xml, xelement, reason...","[deep, copi, element, linq, xml]"
49996,Steps to make a LED blink from a C/C++ program?,What are the easiest steps to make a small cir...,<c++><c>,"[easiest, steps, make, small, circuit, LED, fl...","[Steps, LED, blink, program]",easiest steps make small circuit LED flash pro...,Steps LED blink program,"[easiest, step, make, small, circuit, led, fla...","[step, led, blink, program]"
49997,Passing List<> to SQL Stored Procedure,I've often had to load multiple items to a par...,<c#><sql><sql-server>,"[often, load, multiple, items, particular, rec...","[Passing, List, Stored, Procedure]",often load multiple items particular record da...,Passing List Stored Procedure,"[often, load, multipl, item, particular, recor...","[pas, list, store, procedur]"


In [5]:
df['question'] = df['SL_Body'] + df['SL_Title']
df['question'] = df['question'].apply(' '.join)
df['Tags'] = df['Tags'].str.strip('<>').str.split('><')
mlb = MultiLabelBinarizer()

# Apply MultiLabelBinarizer to 'Tags' column
tags_df = pd.DataFrame(mlb.fit_transform(df['Tags']), columns=mlb.classes_, index=df.index)

# Concatenate original DataFrame with tags DataFrame
df = pd.concat([df, tags_df], axis=1)

In [6]:
X = df.question
Y = tags_df

# Train/Test split

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(list(X), Y, test_size=0.33, random_state=42)

# TfidfVectorizer

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Finding best parameters

In [9]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
# classifier
clf = MultiOutputClassifier(LinearSVC(),n_jobs=-1)
#parameters
param = {'estimator__C':[1,5,10],'estimator__loss':['hinge','squared_hinge'],'estimator__tol':[1e-5,1e-6,1e-7]}
# gridsearch cv
grid_clf = GridSearchCV(clf, param)
grid_clf.fit(X_train,y_train)

GridSearchCV(estimator=MultiOutputClassifier(estimator=LinearSVC(), n_jobs=-1),
             param_grid={'estimator__C': [1, 5, 10],
                         'estimator__loss': ['hinge', 'squared_hinge'],
                         'estimator__tol': [1e-05, 1e-06, 1e-07]})

In [10]:
pd.DataFrame(grid_clf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_estimator__C,param_estimator__loss,param_estimator__tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.596616,1.407791,0.221266,0.006557,1,hinge,0.00001,"{'estimator__C': 1, 'estimator__loss': 'hinge'...",0.432130,0.419975,0.411318,0.424831,0.412162,0.420083,0.007839,10
1,0.966929,0.037874,0.208786,0.008211,1,hinge,0.000001,"{'estimator__C': 1, 'estimator__loss': 'hinge'...",0.432130,0.419975,0.411318,0.424831,0.412162,0.420083,0.007839,10
2,1.120927,0.044055,0.219311,0.001353,1,hinge,0.0,"{'estimator__C': 1, 'estimator__loss': 'hinge'...",0.432130,0.419975,0.411318,0.424831,0.412162,0.420083,0.007839,10
3,0.851440,0.023175,0.222972,0.006820,1,squared_hinge,0.00001,"{'estimator__C': 1, 'estimator__loss': 'square...",0.439308,0.427998,0.415752,0.427998,0.413640,0.424939,0.009351,4
4,0.890886,0.013090,0.222616,0.006740,1,squared_hinge,0.000001,"{'estimator__C': 1, 'estimator__loss': 'square...",0.439308,0.427998,0.415752,0.427998,0.413640,0.424939,0.009351,4
5,1.008907,0.091058,0.228676,0.020076,1,squared_hinge,0.0,"{'estimator__C': 1, 'estimator__loss': 'square...",0.439308,0.427998,0.415752,0.427998,0.413640,0.424939,0.009351,4
6,1.791452,0.034843,0.221690,0.006851,5,hinge,0.00001,"{'estimator__C': 5, 'estimator__loss': 'hinge'...",0.447118,0.433066,0.424198,0.436867,0.430743,0.434398,0.007577,1
7,2.154659,0.036175,0.218164,0.009386,5,hinge,0.000001,"{'estimator__C': 5, 'estimator__loss': 'hinge'...",0.447118,0.433066,0.424198,0.436867,0.430743,0.434398,0.007577,1
8,2.311691,0.077958,0.217313,0.011497,5,hinge,0.0,"{'estimator__C': 5, 'estimator__loss': 'hinge'...",0.447118,0.433066,0.424198,0.436867,0.430743,0.434398,0.007577,1
9,1.370581,0.009686,0.207864,0.005762,5,squared_hinge,0.00001,"{'estimator__C': 5, 'estimator__loss': 'square...",0.436141,0.419764,0.416596,0.428421,0.420397,0.424264,0.007104,7


In [11]:
grid_clf.best_params_

{'estimator__C': 5, 'estimator__loss': 'hinge', 'estimator__tol': 1e-05}

# Using the best parameters

In [12]:
X_train, X_test, y_train, y_test = train_test_split(list(X), Y, test_size=0.33, random_state=42)

In [14]:
from sklearn.pipeline import Pipeline
classifier = MultiOutputClassifier(LinearSVC(C= 5, loss='hinge', tol= 1e-05),n_jobs=-1)
prediction_pipeline = Pipeline([('tf_idf',TfidfVectorizer()),('model',classifier)])
prediction_pipeline.fit(X_train,y_train)

Pipeline(steps=[('tf_idf', TfidfVectorizer()),
                ('model',
                 MultiOutputClassifier(estimator=LinearSVC(C=5, loss='hinge',
                                                           tol=1e-05),
                                       n_jobs=-1))])

In [15]:
prediction = prediction_pipeline.predict(X_test)

In [20]:
import sklearn.metrics as metrics
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import jaccard_score

f1 = []
accuracy = []
precision = []
recall = []
jaccard = []

f1.append(f1_score(y_test,prediction,average='weighted'))
accuracy.append(accuracy_score(y_test,prediction))
precision.append(precision_score(y_test,prediction,average='weighted'))
recall.append(recall_score(y_test,prediction,average='weighted'))
jaccard.append(jaccard_score(y_test, prediction, average='weighted'))  # Compute Jaccard score

mymodel = pd.DataFrame()
mymodel['f1']=f1
mymodel['accuracy']=accuracy
mymodel['precision']=precision
mymodel['recall'] = recall
mymodel['jaccard'] = jaccard

In [21]:
mymodel

,f1,accuracy,precision,recall,jaccard
0,0.664096,0.454865,0.733089,0.610049,0.515688


# streamlit